In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import pickle
import numpy as np
from itertools import combinations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

In [ ]:
full_dictionary_location = "words_250000_train.txt"
full_dictionary = build_dictionary(full_dictionary_location)
print(len(full_dictionary), type(full_dictionary), full_dictionary[:200])

227300 <class 'list'> ['aaa', 'aaaaaa', 'aaas', 'aachen', 'aaee', 'aag', 'aahed', 'aahs', 'aal', 'aalesund', 'aaliis', 'aalst', 'aam', 'aandahl', 'aao', 'aapss', 'aar', 'aarau', 'aardvark', 'aardwolf', 'aaren', 'aargh', 'aarika', 'aaronic', 'aaronite', 'aaronsburg', 'aarp', 'aarrghh', 'aas', 'aasvogels', 'aaup', 'aavso', 'aba', 'ababdeh', 'abac', 'abacas', 'abacaxi', 'abaci', 'abacination', 'abaciscus', 'aback', 'abaco', 'abacterial', 'abactinally', 'abactor', 'abaculus', 'abacuses', 'abada', 'abaddon', 'abadengo', 'abadite', 'abaft', 'abagail', 'abailard', 'abaised', 'abaisse', 'abaka', 'abakas', 'abalation', 'abalienated', 'abalienation', 'abalones', 'abamp', 'abamperes', 'abana', 'abandon', 'abandoned', 'abandonee', 'abandoners', 'abandonment', 'abandons', 'abanet', 'abanic', 'abantes', 'abaptiston', 'abarambo', 'abaris', 'abarticular', 'abas', 'abased', 'abasedness', 'abasements', 'abasers', 'abasgi', 'abashed', 'abashedness', 'abashing', 'abashlessly', 'abashments', 'abasias', 'ab

In [ ]:
chars = set("abcdefghijklmnopqrstuvwxyz.$")
characterset = set('abcdefghijklmnopqrstuvwxyz')
char_to_int = {char: i for i, char in enumerate(sorted(chars))}
int_to_char = {i: char for char, i in char_to_int.items()}

In [ ]:
x_train = []
y_train = []

x_train_wrong_guesses = []
wrong_guess = []
for word in full_dictionary:
    word_length = len(word)
    unique_chars=set(word)
    unique_chars_length = len(set(word))
    for i in [1, 2, unique_chars_length-1]:
      if i < unique_chars_length:
        comb = list(combinations(unique_chars, i))

        for value in comb:
          temp_word=""
          num_of_wrong_guess = random.randint(0, 5)
          wrong_guess.append(random.sample(characterset-set(word), num_of_wrong_guess))
          for char in word:
            if char  in value:
              temp_word += char
            else:
              temp_word += '.'

          x_train.append(temp_word)
          y_train.append(word)

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenized_data = []
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.config.max_length = 50

# for input_text, target_letter in train_data:
#     input_ids = tokenizer.encode(input_text, add_special_tokens=False)  # Encode input text
#     target_id = tokenizer.convert_tokens_to_ids(target_letter)  # Convert target letter to token ID
    # tokenized_data.append({"input_ids": input_ids, "target_id": target_id})

        # print("length of xtrain", len(x_train), i, len(comb))

      # for i in set(word):
      #   temp_word = ''
      #   num_of_wrong_guess = random.randint(0, 5)
      #   wrong_guess.append(random.sample(characterset-set(word), num_of_wrong_guess))
      #   for j in word:
      #     if j != i:
      #       temp_word += j
      #     else:
      #       temp_word += '.'
      #   x_train.append(temp_word)
      #   y_train.append(word)



      # iterations = word_length
      # for _ in range(iterations):
      #     num_of_characters_to_reveal = random.randint(1, len(word)-1) if len(word)-1 >0 else random.randint(0, len(word))
      #     num_of_wrong_guess = random.randint(0, 5)
      #     wrong_guess.append(random.sample(characterset-set(word), num_of_wrong_guess))
      #     chars_to_reveal = random.sample(set(word), num_of_characters_to_reveal)
      #     temp_word = ""
      #     shown_word = chars_to_reveal
      #     for i in range(len(word)):
      #         if  word[i] not in shown_word:
      #             temp_word = temp_word+"."
      #         else:
      #             temp_word = temp_word+word[i]

      #     x_train.append(temp_word)
      #     y_train.append(word)
print(x_train[:10], y_train[:10], len(wrong_guess))

<ipython-input-5-d49dc1a483d0>:17: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  wrong_guess.append(random.sample(characterset-set(word), num_of_wrong_guess))


['...', '......', '...s', 'aaa.', '...s', 'aaa.', '....e.', '..c...', '...h..', 'aa....'] ['aaa', 'aaaaaa', 'aaas', 'aaas', 'aaas', 'aaas', 'aachen', 'aachen', 'aachen', 'aachen'] 9188218


In [ ]:
x_train_encoded = [[char_to_int[char] for char in seq] for seq in x_train]
y_train_encoded = [[char_to_int[char] for char in seq] for seq in y_train]

x_train_encoded_wrong_guess = [[char_to_int[char] for char in seq] for seq in wrong_guess]
print(len(x_train_encoded),"tesjnajnd", y_train_encoded[:10])

9188218 tesjnajnd [[2, 2, 2], [2, 2, 2, 2, 2, 2], [2, 2, 2, 20], [2, 2, 2, 20], [2, 2, 2, 20], [2, 2, 2, 20], [2, 2, 4, 9, 6, 15], [2, 2, 4, 9, 6, 15], [2, 2, 4, 9, 6, 15], [2, 2, 4, 9, 6, 15]]


In [ ]:
max_seq_length = 45
x_train_padded = np.array([seq + [char_to_int["$"]] * (max_seq_length - len(seq)) for seq in x_train_encoded])
y_train_padded = np.array([seq + [char_to_int["$"]] * (50 - len(seq)) for seq in y_train_encoded])


max_wrong_guess_length = 5
x_train_wrong_guess_padded =np.array([seq + [char_to_int["$"]] * (max_wrong_guess_length - len(seq)) for seq in x_train_encoded_wrong_guess])
print(x_train_wrong_guess_padded.shape)

(9188218, 5)


In [ ]:
final_x_train_padded = np.concatenate((x_train_padded,x_train_wrong_guess_padded ), axis=1)
print(final_x_train_padded.shape)
print(y_train_padded.shape)


(9188218, 50)
(9188218, 50)


In [ ]:
# Build the LSTM model
vocab_size = len(chars)
embedding_dim = 55  # Dimension of character embeddings
hidden_units = 5000  # Number of units in the LSTM layer


In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_seq_length + max_wrong_guess_length),  # Adjust input_length
    Bidirectional(LSTM(hidden_units, return_sequences=True)),
    Dense(vocab_size, activation='softmax')
])


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(final_x_train_padded, y_train_padded, epochs=1, batch_size=512)
# model.fit(data_generator, epochs=10)
model.save("hangman_trained_model_1_2_single_character_highfit_bidirectinal.h5")

17946/17946 [==============================] - 6309s 351ms/step - loss: 0.3242 - accuracy: 0.8967


In [ ]:
x_test = '.....a..a...a....'
guessed_chars = 'e'
x_test_encoded = [char_to_int[char] for char in x_test]
guessed_chars_encoded = [char_to_int[char] for char in guessed_chars]
# guessed_chars_encoded = []
x_test_padded = np.array([x_test_encoded + [char_to_int["$"]] * (45 - len(x_test_encoded))])
guessed_chars_padded = np.array([guessed_chars_encoded + [char_to_int["$"]] * (5 - len(guessed_chars_encoded))])
# print(x_test_padded.shape, guessed_chars_padded.shape)
final_padded = np.concatenate((x_test_padded, guessed_chars_padded), axis=1)
predicted_probs = model.predict(final_padded)[0]


already_selected_values = list(x_test)
already_selected_values.extend(list(guessed_chars))

print(already_selected_values)

character_selected = ''
probability_of_selected_value = 0
for index, value in enumerate(x_test):
  if value == '.':

    # if int_to_char[np.argmax(predicted_probs[index])] not in already_selected_values:
    #   if probability_of_selected_value < np.max(predicted_probs[index]):
    #     character_selected = int_to_char[np.argmax(predicted_probs[index])]
    #     probability_of_selected_value= np.max(predicted_probs[index])

    # else:
    while(True):
      print("while running")
      if int_to_char[np.argmax(predicted_probs[index])] in already_selected_values:
        print("changing",int_to_char[np.argmax(predicted_probs[index])])
        predicted_probs[index][np.argmax(predicted_probs[index])] = 0
      else:
        if probability_of_selected_value < np.max(predicted_probs[index]):
          character_selected = int_to_char[np.argmax(predicted_probs[index])]
          probability_of_selected_value= np.max(predicted_probs[index])
          print(index)
        break

print("selected character is",character_selected)



# print(predicted_probs.shape)
# print(int_to_char[np.argmax(predicted_probs[1])])
# predicted_indices = np.argmax(predicted_probs, axis= 1)
# characters_predicted=[int_to_char[index] for index in predicted_indices]
# print(characters_predicted)
# max_predicted_prob_for_each_axis = np.max(predicted_probs, axis=1)
# print(max_predicted_prob_for_each_axis)

# character_predicted = 'a'
# prob_till_now = 0
# for index,value in enumerate(characters_predicted[:len(x_test)]):
#     if max_predicted_prob_for_each_axis[index] > prob_till_now and value in characterset and value not in guessed_chars and value not in x_test:
#       character_predicted = value
# print("character_predicted_end", character_predicted)

# print("predicted_probs", predicted_probs)
# predicted_probs = predicted_probs[0]
# print(np.argmax(predicted_probs))
# predicted_next_letter = int_to_char[np.argmax(predicted_probs)]
# print(predicted_next_letter)


1/1 [==============================] - 1s 711ms/step
['.', '.', '.', '.', '.', 'a', '.', '.', 'a', '.', '.', '.', 'a', '.', '.', '.', '.', 'e']
while running
0
while running
1
while running
while running
while running
4
while running
while running
while running
while running
10
while running
while running
while running
while running
15
while running
16
selected character is n


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
